In [1]:
import instaloader
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
USER = os.getenv('IG_USER')
PASSWORD = os.getenv('IG_PASS')

server = os.getenv('DB_SERVER')
database = os.getenv('DB_DATABASE')
username = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')

connection_string = f'mysql+pymysql://{username}:{password}@{server}/{database}'

engine = create_engine(connection_string)

seguidos = []
seguidores = []

In [3]:
# Instancia en L
L = instaloader.Instaloader()

# Inicio de sesión
L.login(USER, PASSWORD)

# Recopila datos del perfil
profile = instaloader.Profile.from_username(L.context, USER)

In [4]:
# Itera seguidos
print("{} follows these profiles:".format(profile.username))
for followee in profile.get_followees():
    seguidos.append(followee.username)

ssala_ follows these profiles:


In [5]:
# Itera seguidores
print("{} follows these profiles:".format(profile.username))
for followers in profile.get_followers():
    seguidores.append(followers.username)

ssala_ follows these profiles:


In [6]:
# Crea el DF de seguidos para subir a la base de datos
data = {'Usuario': seguidos}
df = pd.DataFrame(data)
with engine.begin() as connection:
    delete_sql = text("""
        DELETE FROM seguidos
        """)
    connection.execute(delete_sql)
    
df.to_sql('seguidos', engine, if_exists='append', index=False)

In [7]:
# Crea el DF de seguidores para subir a la base de datos
data = {'Usuario': seguidores}
df = pd.DataFrame(data)
with engine.begin() as connection:
    delete_sql = text("""
        DELETE FROM seguidores
        """)
    connection.execute(delete_sql)
    
df.to_sql('seguidores', engine, if_exists='append', index=False)

In [8]:
# Convertir las listas en sets
set_seguidos = set(seguidos)
set_seguidores = set(seguidores)

# Usuarios que sigues pero que no te siguen
no_te_siguen = set_seguidos - set_seguidores

# Usuarios que te siguen pero que tú no sigues
no_sigues = set_seguidores - set_seguidos

# Convertir los resultados de vuelta a listas (opcional)
no_te_siguen_lista = list(no_te_siguen)
no_sigues_lista = list(no_sigues)

In [9]:
# Crear DataFrames a partir de los diccionarios
df_seguidores = pd.DataFrame({
    'Seguidor No Seguido': no_sigues_lista
})

df_seguidos = pd.DataFrame({
    'Seguido que No Sigue': no_te_siguen_lista
})

# Guardar los DataFrames en archivos Excel
df_seguidores.to_excel('seguidores.xlsx', index=False)
df_seguidos.to_excel('seguidos.xlsx', index=False)